# 基于Transformers的命名实体识别

## 1.导包

In [1]:
from transformers import AutoTokenizer,AutoModelForTokenClassification,TrainingArguments,Trainer,DataCollatorForTokenClassification
import evaluate
from datasets import load_dataset
import torch
from torch.utils.data import Dataset,DataLoader

## 2.加载数据集

In [2]:
ner_dataset = load_dataset("lansinuote/peoples-daily-ner")
ner_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})

In [3]:
ner_dataset["train"]

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 20865
})

In [4]:
ner_dataset["train"][0]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}

In [5]:
ner_dataset["train"].features

{'id': Value('string'),
 'tokens': List(Value('string')),
 'ner_tags': List(ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']))}

In [6]:
ner_dataset["train"].features["ner_tags"]

List(ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']))

In [7]:
ner_dataset["train"].features["ner_tags"].feature

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'])

In [8]:
label_list = ner_dataset["train"].features["ner_tags"].feature.names
label_list # 最后会有一步文本映射

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

## 3.数据集预处理

In [9]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

In [10]:
to = tokenizer(ner_dataset["train"][0]["tokens"], is_split_into_words=True) # 将拆分的词组合回一句话
to # 测试用，不用于主代码

{'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
to.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, None]

In [12]:
ner_dataset["train"]["ner_tags"][0]

[0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]

In [13]:
def process_function(examples):
    tokenizer_examples = tokenizer(examples["tokens"], max_length=128, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]): # examples["ner_tags"] = ner_dataset["train和validation和test"]["ner_tags"],
        word_ids = tokenizer_examples.word_ids(batch_index=i) # 相当于前文的to.word_ids()
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100) # -100表示忽略
            else:
                label_ids.append(label[word_id]) # 去除None
        labels.append(label_ids)
    tokenizer_examples["labels"] = labels
    return tokenizer_examples

In [14]:
tokenizer_dataset = ner_dataset.map(process_function, batched=True)
tokenizer_dataset

Map:   0%|          | 0/4637 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4637
    })
})

In [15]:
print(tokenizer_dataset["train"][0])

{'id': '0', 'tokens': ['海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间', '的', '海', '域', '。'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0], 'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100]}


## 4.创建模型

In [22]:
model = AutoModelForTokenClassification.from_pretrained("hfl/chinese-macbert-base", num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
model.config.id2label = {idx: label for idx, label in enumerate(label_list)}
model.config

BertConfig {
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "dtype": "float32",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type"

    ## 5.创建评估函数

In [25]:
seqeval = evaluate.load("seqeval")
seqeval

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': List(Value('string')), 'references': List(Value('string'))}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of shape (n_samples,), weights for individual samples. default: None
    zero_division: Which value to substitute as a

In [30]:
import numpy as np
def eval_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)

    true_predictions = [[label_list[p] for p, l in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    true_labels = [[label_list[l] for p, l in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = seqeval.compute(predictions=true_predictions, references=true_labels, mode="strict", scheme="IOB2")
    return {
        "f1": results["overall_f1"]
    }


## 6.配置训练参数

In [34]:
args = TrainingArguments(
    output_dir="models_for_ner",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    logging_steps=50,
    num_train_epochs=3
)

## 7.创建训练器

In [35]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenizer_dataset["train"],
    eval_dataset=tokenizer_dataset["validation"],
    compute_metrics=eval_metric,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)
)

## 8.模型训练

In [36]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.027300,0.021337,0.945560
2,0.012800,0.017282,0.951303
3,0.006400,0.017926,0.957689


TrainOutput(global_step=981, training_loss=0.02451849159965943, metrics={'train_runtime': 14516.0382, 'train_samples_per_second': 4.312, 'train_steps_per_second': 0.068, 'total_flos': 3940951205762142.0, 'train_loss': 0.02451849159965943, 'epoch': 3.0})

In [38]:
trainer.evaluate(eval_dataset=tokenizer_dataset["test"])

{'eval_loss': 0.023530589416623116,
 'eval_f1': 0.9467487289269467,
 'eval_runtime': 178.0055,
 'eval_samples_per_second': 26.05,
 'eval_steps_per_second': 0.208,
 'epoch': 3.0}

## 9.模型预测

In [39]:
from transformers import pipeline

In [49]:
ner_pipe = pipeline("token-classification",model=model,tokenizer=tokenizer,device=0,aggregation_strategy="simple")
res = ner_pipe("我叫张三，我今年18岁，我住在上海。")
res

Device set to use cuda:0


[{'entity_group': 'PER',
  'score': np.float32(0.9994254),
  'word': '张 三',
  'start': 2,
  'end': 4},
 {'entity_group': 'LOC',
  'score': np.float32(0.9994155),
  'word': '上 海',
  'start': 15,
  'end': 17}]

In [50]:
ner_result = {}
x = "我叫张三，我今年18岁，我住在上海。"
for r in res:
    if r['entity_group'] not in ner_result:
        ner_result[r['entity_group']] = []
    ner_result[r['entity_group']].append(x[r['start']:r['end']])
ner_result

{'PER': ['张三'], 'LOC': ['上海']}